In [1]:
import pandas as pd

In [2]:
dados_tratados = pd.read_csv('dados_tratados.csv', encoding='latin-1', sep=';')

In [29]:
dados_tratados.head()

,DS_ELEICAO,SG_UF,DS_CARGO,NM_URNA_CANDIDATO,SG_PARTIDO,QT_VOTOS_NOMINAIS_VALIDOS,DS_SIT_TOT_TURNO
0,ELEIÇÕES GERAIS ESTADUAIS 2022,CE,Deputado Federal,THIAGO ESMERALDO,PP,0,SUPLENTE
1,ELEIÇÕES GERAIS ESTADUAIS 2022,CE,Deputado Federal,THIAGO ESMERALDO,PP,2,SUPLENTE
2,ELEIÇÕES GERAIS ESTADUAIS 2022,CE,Deputado Federal,CORONEL AGINALDO,PL,429,SUPLENTE
3,ELEIÇÕES GERAIS ESTADUAIS 2022,CE,Deputado Estadual,ADRIANNO IMPERADOR,UNIÃO,2,SUPLENTE
4,ELEIÇÕES GERAIS ESTADUAIS 2022,CE,Deputado Estadual,ADRIANNO IMPERADOR,UNIÃO,0,SUPLENTE


#### Muitas linhas com informações duplicadas, então agruparei os semelhantes e somarei os votos

In [9]:
colunas = ['DS_ELEICAO', 'SG_UF', 'DS_CARGO', 'NM_URNA_CANDIDATO', 'SG_PARTIDO', 'DS_SIT_TOT_TURNO']

In [51]:
dado_agrupados_por_nome = pd.DataFrame(dados_tratados.groupby(colunas)['QT_VOTOS_NOMINAIS_VALIDOS'].sum().reset_index())

In [52]:
dado_agrupados_por_nome.head()

,DS_ELEICAO,SG_UF,DS_CARGO,NM_URNA_CANDIDATO,SG_PARTIDO,DS_SIT_TOT_TURNO,QT_VOTOS_NOMINAIS_VALIDOS
0,ELEIÇÃO CONSELHO DISTRITAL 2022,PE,Conselheiro Distrital,AILTON JUNIOR,P41,ELEITO,394
1,ELEIÇÃO CONSELHO DISTRITAL 2022,PE,Conselheiro Distrital,ARTUR CÂNDIDO,P59,NÃO ELEITO,78
2,ELEIÇÃO CONSELHO DISTRITAL 2022,PE,Conselheiro Distrital,BETO FAUSTINO,P74,NÃO ELEITO,9
3,ELEIÇÃO CONSELHO DISTRITAL 2022,PE,Conselheiro Distrital,CARINE DINO,P78,ELEITO,112
4,ELEIÇÃO CONSELHO DISTRITAL 2022,PE,Conselheiro Distrital,CARLOS FLOR,P87,NÃO ELEITO,34


#### Mesmo agrupando por nome, os candidatos à presidencia ainda aparecem duplicados, pois tiveram votos em multiplos estados (28)
#### 28 estados, pois, na coluna [SG_UF], o estado 'ZZ' representa, na verdade, votos no exterior.
#### E no caso da candidata Vera, aparece 29 vezes, pois 28 registros são referentes à candidata à presidencia Vera Lúcia e 1 para uma homonima candiata à deputado estadual

In [18]:
dado_agrupados_por_nome['NM_URNA_CANDIDATO'].value_counts()

VERA                   29
SIMONE TEBET           28
CONSTITUINTE EYMAEL    28
FELIPE D AVILA         28
JAIR BOLSONARO         28
                       ..
PAULA DO SOCIAL         1
PAULA BORGES            1
PAULA BATALHA           1
PATRÍCIA RAMOS          1
VILELA DO PT            1
Name: NM_URNA_CANDIDATO, Length: 25642, dtype: int64

#### Criando um filtro, para verificar se além de candidatos à presidencia, algum outro candidato aparece duplicado

In [20]:
dado_agrupados_por_nome.DS_CARGO.unique()

array(['Conselheiro Distrital', 'Presidente', 'Deputado Estadual',
       'Deputado Federal', 'Governador', 'Senador', 'Deputado Distrital'],
      dtype=object)

In [24]:
candidados_sem_presidente = dado_agrupados_por_nome.query('DS_CARGO !="Presidente"')

In [26]:
candidados_sem_presidente['NM_URNA_CANDIDATO'].value_counts()

LUIZ CARLOS        6
ANA PAULA          6
JOÃO PAULO         5
ZÉ CARLOS          5
ANA CRISTINA       5
                  ..
PAULA DO SOCIAL    1
PAULA BORGES       1
PAULA BATALHA      1
PATRÍCIA RAMOS     1
VILELA DO PT       1
Name: NM_URNA_CANDIDATO, Length: 25632, dtype: int64

#### Como houveram alguns candidatos que apareceram duplicados, verifiquei se não se trata apenas de homonimos. O resultado se mostrou positivo, são apenas pessoas com os mesmos nomes, portanto, o filtro foi bem aplicado.

In [59]:
candidados_sem_presidente.query('NM_URNA_CANDIDATO == "LUIZ CARLOS"')

,DS_ELEICAO,SG_UF,DS_CARGO,NM_URNA_CANDIDATO,SG_PARTIDO,DS_SIT_TOT_TURNO,QT_VOTOS_NOMINAIS_VALIDOS
2211,ELEIÇÕES GERAIS ESTADUAIS 2022,AP,Deputado Federal,LUIZ CARLOS,PSDB,NÃO ELEITO,11981
10107,ELEIÇÕES GERAIS ESTADUAIS 2022,MG,Deputado Federal,LUIZ CARLOS,UNIÃO,SUPLENTE,2098
10775,ELEIÇÕES GERAIS ESTADUAIS 2022,MS,Deputado Estadual,LUIZ CARLOS,PSDB,SUPLENTE,89
11476,ELEIÇÕES GERAIS ESTADUAIS 2022,MT,Deputado Federal,LUIZ CARLOS,AGIR,NÃO ELEITO,230
13429,ELEIÇÕES GERAIS ESTADUAIS 2022,PE,Deputado Estadual,LUIZ CARLOS,UNIÃO,SUPLENTE,2165
19834,ELEIÇÕES GERAIS ESTADUAIS 2022,RR,Deputado Estadual,LUIZ CARLOS,PDT,NÃO ELEITO,17


In [43]:
candidados_sem_presidente['DS_SIT_TOT_TURNO'].unique()

array(['ELEITO', 'NÃO ELEITO', 'ELEITO POR QP', 'SUPLENTE',
       'ELEITO POR MÉDIA', '2º TURNO'], dtype=object)

#### Por fim, somente uma checagem do número de eleitos para cada cargo no 1º turno, comparando com dados divulgadas em sites de notícias. Os números bateram.

In [45]:
len(candidados_sem_presidente.query(
    'DS_CARGO == "Deputado Federal" & (DS_SIT_TOT_TURNO == "ELEITO" | DS_SIT_TOT_TURNO == "ELEITO POR QP" | DS_SIT_TOT_TURNO == "ELEITO POR MÉDIA")'))

513

In [46]:
len(candidados_sem_presidente.query(
    'DS_CARGO == "Senador" & (DS_SIT_TOT_TURNO == "ELEITO" | DS_SIT_TOT_TURNO == "ELEITO POR QP" | DS_SIT_TOT_TURNO == "ELEITO POR MÉDIA")'))

27

In [53]:
len(dado_agrupados_por_nome.query(
    'DS_CARGO == "Deputado Estadual" & (DS_SIT_TOT_TURNO == "ELEITO" | DS_SIT_TOT_TURNO == "ELEITO POR QP" | DS_SIT_TOT_TURNO == "ELEITO POR MÉDIA")'))

1035

In [54]:
len(dado_agrupados_por_nome.query(
    'DS_CARGO == "Governador" & (DS_SIT_TOT_TURNO == "ELEITO" | DS_SIT_TOT_TURNO == "ELEITO POR QP" | DS_SIT_TOT_TURNO == "ELEITO POR MÉDIA")'))

15

#### Por fim, exportando a base de dados tratada, para futura análise.

In [55]:
dado_agrupados_por_nome.to_csv('dados_agrupados_por_nome.csv', index=False, encoding='latin-1', sep=';')